In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib widget
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt

from scwidgets import (AnswerRegistry, TextareaAnswer, CodeDemo,
                       ParametersBox, PyplotOutput, ClearedOutput,
                       AnimationOutput,CheckRegistry)

from widget_code_input import WidgetCodeInput
from ipywidgets import Layout, Output, Textarea

from ase import Atoms

Before you work on this module, please input your name to create a file that will store your answers, or load an existing file if you need to continue.

In [2]:
check_registry = CheckRegistry()  # this is needed to coordinate code checking
answer_registry = AnswerRegistry(prefix="module_xx")
display(answer_registry)

AnswerRegistry(children=(Output(layout=Layout(height='99%', width='99%')), HBox(children=(Dropdown(description…

# A brief Python re-cap

This course assumes basic knowledge of the Python language (variables, conditionals, lists, function definitions...). If you are not familiar with these concepts, or with the syntax that is used to manipulate them in Python, you should follow some online crash-course, e.g. **XXXXX give some links**.

In this module we focus on three concepts that some may not be familiar with, and that are used heavily in this class.

* Modules, and how to import functionalities from external packages
* Python objects: variables and methods, from a user perspective
* Numpy arrays: numerical lists for maths, indexing and operations

# Extending Python through modules

One of the features that made Python so successful is how easy it is to extend its capabilities by including external libraries into a program.  

<p style="text-align:center">
<img src="./figures/xkcd-python.png"/><br>
    <a href="https://xkcd.com/353/">Reproduced from xkcd.com</a>
    Licensed under <a href="https://creativecommons.org/licenses/by-nc/2.5/">CC-BY-NC-2.5</a>
</p>

There are several ways to include features from external packages, and it is important to understand what are the implications for _using_ those features. For example, let's consider the sine function `sin`. Python does not have built-in trigonometric functions, so we need to import them from and external library that implements the sine function, for example the `math` package.

The simplest way to include functionality from a package is to import the whole library, using the syntax 
```python 
import library as alias
``` 
The `as alias` part is optional and allows to provide a shorthand for when a library is used a lot or has a very long name. In order to access the functions provided by `library`, it is then necessary to specify the `alias`, too, e.g. `alias.function(1)`. If `alias` is not given, one simply uses `library.function(1)`. This is because the functions are not brought into the main namespace, but remain part of the library. For example,

```python
import math
print(math.sin(1.0))
```

It is also possible to import just a few selected functions, and bring them into the main namespace. This is useful when only a few of the functionalities offered by `library` need to be used a lot. The syntax is then `from library import function`, for example
```python
from math import sin
print(sin(1.0))
```
It is also prossible to bring in _all_ functions from library into the main namespace using `from library import *`, but this is discoruaged, as it increases the risk of having clashes between the names of functions imported from different packages. 

In [38]:
# initialize a widget for code input
ex01_wci = WidgetCodeInput(
        function_name = "sin_cos", 
        function_parameters="x",
        code_theme = "default",
        docstring="""
Computes the sine and the cosine of an input value, and return both as a tuple

:param x: the input angle, in radians

:return: sin(x), cos(x)
""",
            function_body="""
# write code to import the sine and cosine function from the `math` module


# compute sine and cosine
s = 0
c = 1

return s, c
"""
)

# initialize a CodeDemo object and initialize its checks in the check_registry:
ex01_demo = CodeDemo( 
            code_input = ex01_wci,
            check_registry = check_registry,
            ) 

check_registry.add_check(ex01_demo,
                         inputs_parameters=[{"x" : 0}, {"x":np.pi}, {"x": 0.12345} ],
                         reference_outputs=[(0.0, 1.0), (0, -1.0), (0.12313667785133202, 0.9923897211114882)],
                         equal=np.allclose)
# Registers the answer
answer_registry.register_answer_widget("ex01_function", ex01_demo)

In [39]:
# helper function to generate values for checking the function
check_registry.print_reference_outputs(ex01_demo)

Check 0:
[(0, 1), (0, 1), (0, 1)]


<span style="color:blue"> **01** Write a function that imports the sine and cosine functions from the `math` library, and returns simultaneously sine and cosine of the argument. </span>

_NB: it is possible to import python packages also within the body of a function. You can try different ways of importing the functions._

In [41]:
display(ex01_demo)

CodeDemo(children=(HBox(children=(CodeDemoBox(_dom_classes=('scwidget-box', 'scwidget-box--unchecked')), VBox(…

# Python objects, from a user perspective

This section provides an operative guide to _using_ Python classes, assuming no formal training in object-oriented programming. If you want to learn more (e.g. if you want to see how you can define your own objects) you can follow some online tutorials, e.g. **add a few links here**.

An object - in Python and elsewhere - is a code construct that holds both data (_member variables_) and functionality (_methods_). Combining data and code makes it possible to conceptualize the relationship between the attributes of an item, and the operations that can be performed with (or on) it, and is usually thought to lead to code that is clearer, easier to understand and maintain. Objects in Python can be used like any other variable - passed as arguments to a function, and returned as the output of a function call. 

Consider for example an object which is meant to represent a triangle. The geometry of a triangle can be entirely defined by its three sides, so we can combine in the same structure three floating point numbers describing the sides, `a, b, c`.  It would make sense to ask to compute the area of the triangle, or to scale it by a constant factor. A typical use case would be as follows

```python
my_triangle = Triangle(a=3.0, b=4.0, c=5.0)
print(f"The triangle has sides a={my_triangle.a}, b={my_triangle.b}, c={my_triangle.c}")
print(f"The area of the triangle is {my_triangle.area()}")
print(f"Now scaling the triangle...")
my_triangle.scale(factor=2.0)
print(f"The triangle has sides a={my_triangle.a}, b={my_triangle.b}, c={my_triangle.c}")
print(f"The area of the triangle is {my_triangle.area()}")
```

## Initialization

Let's look at this code in some more detail. The first line corresponds to the _initialization_ of the `Triangle` object. `Triangle` is the name of the _class_ that defines the object (usually this will have been imported by a library), and by calling it with tree arguments, corresponding to the length of the triangle sides, we create an _instance_ of a triangle with the desired values. One can create many instances of a class, and each can be manipulated independently

```python
triangle_one = Triangle(a=3.0, b=4.0, c=2.0)
triangle_two = Triangle(a=2.0, b=2.0, c=4.0)
```

Once an instance of a class is created, it is possible to interact with it in many ways. One can retrieve the values of the member variables, or directly set their values

```python
print(f"The triangle has sides a={triangle_one.a}, b={triangle_one.b}, c={triangle_one.c}")
triangle_one.c = 5.0
triangle_two.c = 2.0
```


In [25]:
import math
class Triangle:
    def __init__(self, a, b, c):
        self.a, self.b, self.c = a, b, c
    
    def area(self):
        s = (self.a+self.b+self.c)/2       
        return math.sqrt(s*(s-self.a)*(s-self.b)*(s-self.c))
    
    def scale(self, factor):
        self.a*=factor; self.b*=factor; self.c*=factor; 
# ugly but necessary to make Triangle available in the CodeInput 
import builtins
builtins.Triangle = Triangle

In [43]:
# initialize a widget for code input
ex02_wci = WidgetCodeInput(
        function_name = "equilateral_triangle", 
        function_parameters="side",
        code_theme = "default",
        docstring="""
Creates an equilateral triangle given its side

:param side: The side of the triangle

:return: An equilateral `Triangle` object
""",
            function_body="""
# NB: the Triangle class is not a builtin object and is only available here for this specific exercise

# Create ai instance of a Triangle and return it
my_triangle = 0

return my_triangle
"""
)

# initialize a CodeDemo object and initialize its checks in the check_registry:
ex02_demo = CodeDemo( 
            code_input = ex02_wci,
            check_registry = check_registry,
            ) 

def ex02_asserts(output, reference):
    assert isinstance(output, Triangle), "The function should return a `Triangle` object"
    assert (output.a==output.b and output.a==output.c), "The triangle is not equilateral"  
    assert (output.a==reference[0]), "The side of the triangle does not match the input"

check_registry.add_check(ex02_demo,
                         inputs_parameters=[{"side" : 1}, {"side": 2}],
                         fingerprint= lambda t: (t.a, t.b, t.c),
                         reference_outputs=[(1,1,1), (2,2,2)],
                         custom_asserts=ex02_asserts
                         )

answer_registry.register_answer_widget("ex02_function", ex02_demo)

In [42]:
# helper function to generate values for checking the function
check_registry.print_reference_outputs(ex02_demo)

Check 0:
[(1, 1, 1), (2, 2, 2)]


<span style="color:blue"> **02** Write a function that creates a `Triangle` object, initialized to be an equilateral triangle with three equal sides. </span>

_Feel free to experiment setting and checking the values of the instance variables before converging on a correct answer. You can see the outputs by running the checks._

In [45]:
display(ex02_demo)

CodeDemo(children=(HBox(children=(CodeDemoBox(_dom_classes=('scwidget-box', 'scwidget-box--unchecked')), VBox(…

<span style="color:blue"> **01** Write a function that imports the sine and cosine functions from the `math` library, and returns simultaneously sine and cosine of the argument. </span>

_NB: it is possible to import python packages also within the body of a function. You can try different ways of importing the functions._

In [10]:
my_triangle = Triangle(a=3.0, b=4.0, c=5.0)
print(f"The triangle has sides a={my_triangle.a}, b={my_triangle.b}, c={my_triangle.c}")
print(f"The area of the triangle is {my_triangle.area()}")
print(f"Now scaling the triangle...")
my_triangle.scale(factor=2.0)
print(f"The triangle has sides a={my_triangle.a}, b={my_triangle.b}, c={my_triangle.c}")
print(f"The area of the triangle is {my_triangle.area()}")

The triangle has sides a=3.0, b=4.0, c=5.0
The area of the triangle is 6.0
Now scaling the triangle...
The triangle has sides a=6.0, b=8.0, c=10.0
The area of the triangle is 24.0
